# Model 3 : transformers applied to time series forecasting

## Imports and Data pre-processing

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import time
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

torch.manual_seed(0)
np.random.seed(0)

In [4]:
def custom_date_parser(date_string):
    return pd.datetime.strptime(date_string, '%d.%m.%Y %H:%M:%S')

In [5]:
path = '/home/rbazin/ECSE552/ECSE552-HW4/'

In [6]:
data = pd.read_csv(os.path.join(path, "weather_train.csv"), parse_dates = ['Date Time'], date_parser=custom_date_parser)
data.head()

/tmp/ipykernel_824786/649081062.py:2: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  return pd.datetime.strptime(date_string, '%d.%m.%Y %H:%M:%S')


,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,2009-01-01 01:00:00,996.50,-8.05,265.38,-8.78,94.4,3.33,3.14,0.19,1.96,3.15,1307.86,0.21,0.63,192.7
1,2009-01-01 02:00:00,996.62,-8.88,264.54,-9.77,93.2,3.12,2.90,0.21,1.81,2.91,1312.25,0.25,0.63,190.3
2,2009-01-01 03:00:00,996.84,-8.81,264.59,-9.66,93.5,3.13,2.93,0.20,1.83,2.94,1312.18,0.18,0.63,167.2
3,2009-01-01 04:00:00,996.99,-9.05,264.34,-10.02,92.6,3.07,2.85,0.23,1.78,2.85,1313.61,0.10,0.38,240.0
4,2009-01-01 05:00:00,997.46,-9.63,263.72,-10.65,92.2,2.94,2.71,0.23,1.69,2.71,1317.19,0.40,0.88,157.0


In [7]:
data.sort_values(by='Date Time', ascending=True, inplace=True)

In [8]:
data = data.rename(columns={'p (mbar)': 'p', 'T (degC)': 'T', 'rh (%)': 'rh', 'wv (m/s)': 'wv', 'Date Time': 'date'})

In [9]:
data['date'] = data['date'].apply(lambda x: x.replace(minute=0))

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56072 entries, 0 to 56071
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             56072 non-null  datetime64[ns]
 1   p                56072 non-null  float64       
 2   T                56072 non-null  float64       
 3   Tpot (K)         56072 non-null  float64       
 4   Tdew (degC)      56072 non-null  float64       
 5   rh               56072 non-null  float64       
 6   VPmax (mbar)     56072 non-null  float64       
 7   VPact (mbar)     56072 non-null  float64       
 8   VPdef (mbar)     56072 non-null  float64       
 9   sh (g/kg)        56072 non-null  float64       
 10  H2OC (mmol/mol)  56072 non-null  float64       
 11  rho (g/m**3)     56072 non-null  float64       
 12  wv               56072 non-null  float64       
 13  max. wv (m/s)    56072 non-null  float64       
 14  wd (deg)         56072 non-null  float

In [11]:
data_df = data.copy()

## Model architecture

First, we build the architecture :

In [16]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()       
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        # div_term = torch.exp(
        #     torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)
        # )
        div_term = 1 / (10000 ** ((2 * np.arange(d_model)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term[0::2])
        pe[:, 1::2] = torch.cos(position * div_term[1::2])

        pe = pe.unsqueeze(0).transpose(0, 1) # [5000, 1, d_model],so need seq-len <= 5000
        #pe.requires_grad = False
        self.register_buffer('pe', pe)

    def forward(self, x):
        # print(self.pe[:x.size(0), :].repeat(1,x.shape[1],1).shape ,'---',x.shape)
        # dimension 1 maybe inequal batchsize
        return x + self.pe[:x.size(0), :].repeat(1,x.shape[1],1)

In [17]:
class TimeFormer(nn.Module):
    def __init__(self,feature_size=250,num_encoder_layers=1,dropout=0.1, nbr_labels=4):
        super(TimeFormer, self).__init__()
        self.model_type = 'Transformer'
        self.input_embedding  = nn.Linear(nbr_labels, feature_size)
        self.src_mask = None

        self.pos_encoder = PositionalEncoding(feature_size)
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=10, dropout=dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_encoder_layers, norm=None)
        self.decoder = nn.Linear(feature_size * nbr_labels, nbr_labels)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self,src):
        # src with shape (input_window, batch_len, 1)
        if self.src_mask is None or self.src_mask.size(0) != src.shape[1]:
            device = src.device
            mask = self._generate_square_subsequent_mask(src.shape[1]).to(device)
            self.src_mask = mask

        src = self.input_embedding(src) # linear transformation before positional embedding
        src = self.pos_encoder(src) # positional embedding
        output = self.transformer_encoder(src,self.src_mask) # transformer encoder
        output = output.reshape(output.shape[0], -1) # flatten the output
        output = self.decoder(output)  # linear transformation decoder
        return output

    def _generate_square_subsequent_mask(self, size):
        mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask


We create the a dataset class to encapsulate and format our data :

In [34]:
class TimeFormerDataset(Dataset):
    """
    Dataset class used for TimeFormer model.
    """
    def __init__(self, data_df, enc_seq_len=4, window_step=1, labels_as_features=False):
        super().__init__()

        self.enc_seq_len = enc_seq_len
        self.window_step = window_step

        self.labels = torch.from_numpy(data_df.loc[:, ["p", "T", "rh", "wv"]].to_numpy()).type(torch.float32)
        self.features = None
        if not labels_as_features:
            self.features = torch.from_numpy(data_df.loc[:, [k for k in data_df.columns if k not in ["p", "T", "rh", "wv", "date"]]].to_numpy()).type(torch.float32)

        # list of tuple (start, end) of the sequences according to window_step and enc_seq_len
        self.seq_list = [(i, i + enc_seq_len) for i in range(0, len(data_df) - enc_seq_len, window_step)]
    
    def __len__(self):
        return self.labels.shape[0]
    
    def __getitem__(self, idx):
        """ 
        Returns the input and target sequences for a given index.
        """

        # get the start and end of the sequence
        start, end = self.seq_list[idx]

        # get the input sequence
        src_seq = self.features[start:end] if self.features is not None else self.labels[start:end]
        trgt_seq = self.labels[end]

        return src_seq, trgt_seq.squeeze()

In [ ]:
# training function
def train(model, criterion, optimizer, epochs, train_loader, val_loader, device, save_path):
    
    train_losses = []
    val_losses = []

    start = time.time()
    for i, epoch in enumerate(tqdm(range(epochs), desc="Epochs")):

        total_loss = 0.
        model.train()
        for batch in train_loader:  
            
            srcs = batch[0].to(device)
            trgts = batch[1].to(device)

            optimizer.zero_grad()
            output = model(srcs)
            loss = criterion(output, trgts)

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.7)
            optimizer.step()

            total_loss += loss.item()
        
        train_losses.append(total_loss / len(train_loader))

        # validation
        total_loss = 0.
        model.eval()
        with torch.no_grad():
            for batch in val_loader:
                srcs = batch[0].to(device)
                trgts = batch[1].to(device)

                output = model(srcs)
                loss = criterion(output, trgts)

                total_loss += loss.item()
            
            val_losses.append(total_loss / len(val_loader))

        tqdm.write(f"Epoch {epoch+1} | Train loss: {train_losses[-1]:.4f} | Val loss: {val_losses[-1]:.4f}")
        
        # checkpoint
        if i % 10 == 0:
            torch.save(model.state_dict(), save_path)
    
    end = time.time()
    print(f"Training time: {end - start:.2f} seconds")
    
    return model, train_losses, val_losses

In [ ]:
# evaluation function
def evaluate(model, criterion, test_loader, device):
    model.eval()
    with torch.no_grad():
        total_loss = 0.
        for batch in test_loader:
            srcs = batch[0].to(device)
            trgts = batch[1].to(device)

            output = model(srcs)
            loss = criterion(output, trgts)

            total_loss += loss.item()
        
        return total_loss / len(test_loader)

In [44]:
# hyperparameters
epochs = 100
lr = 1e-3
enc_seq_len = 4 # number of past observations to use
window_step = 1 # step between two consecutive windows
nbr_labels = 4 # number of labels to predict
batch_size = 32
dropout = 0.1
num_encoder_layers = 1 # number of transformer encoder layers
feature_size = 250 # size of the feature vector in the input embedding layer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# datasets and dataloaders
time_ds = TimeFormerDataset(data_df, enc_seq_len=enc_seq_len, window_step=window_step, labels_as_features=True)

train_size = int(0.8 * len(time_ds)) # 80% for training
val_size = int(0.1 * len(time_ds)) # 10% for validation
test_size = len(time_ds) - train_size - val_size # remaining for test

train_ds, val_ds, test_ds = random_split(time_ds, [train_size, val_size, test_size])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

In [ ]:
# training 

model = TimeFormer(nbr_labels=nbr_labels, feature_size=feature_size, num_encoder_layers=num_encoder_layers, dropout=dropout).to(device)

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model, train_losses, val_losses = train(model, criterion, optimizer, epochs, train_loader, val_loader, device, "timeformer.pth")

In [ ]:
print(f"Final losses | Train loss: {train_losses[-1]:.4f} | Val loss: {val_losses[-1]:.4f}")

In [ ]:
# plot losses
plt.plot(train_losses, label="Train loss")
plt.plot(val_losses, label="Val loss")
plt.legend()

In [ ]:
test_loss = evaluate(model, criterion, test_loader, device)
print(f"Test loss: {test_loss:.4f}")